In [2]:
import pandas as pd
import gdown
import ipaddress
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler


In [7]:
# Load Fraud_Data, Credit Card Data, and IP-to-Country data
fraud_df = pd.read_csv("/content/Fraud_Data.csv")
creditcard_df = pd.read_csv("/content/creditcard1.csv")
ip_country_df = pd.read_csv("/content/IpAddress_to_Country.csv")


In [8]:
# ============================================================
# STEP 3: Data Cleaning - IP Address to Country Data
# ============================================================

# Convert lower_bound_ip_address to integer (it was a float)
ip_country_df['lower_bound_ip_address'] = ip_country_df['lower_bound_ip_address'].astype(int)


In [9]:
# ============================================================
# STEP 4: Data Cleaning - Fraud Data
# ============================================================

# Remove duplicate rows from fraud_df (if any)
fraud_df.drop_duplicates(inplace=True)

# Check for missing values and print a summary
print("Missing values in Fraud Data:")
print(fraud_df.isna().sum())

# Convert timestamp columns to datetime objects.
# (Assuming columns are named 'signup_time' and 'purchase_time')
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'], errors='coerce')
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'], errors='coerce')


Missing values in Fraud Data:
user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64


In [10]:
# ============================================================
# STEP 5: Convert IP Address Strings to Integer
# ============================================================

# Define a function that converts an IPv4 address (string) to an integer.
def ip_to_int(ip_str):
    try:
        return int(ipaddress.IPv4Address(ip_str))
    except Exception:
        return np.nan

# Apply the conversion to create a new column 'ip_int'
fraud_df['ip_int'] = fraud_df['ip_address'].apply(ip_to_int)


In [11]:
# ============================================================
# STEP 6: Merge Fraud Data with IP-to-Country Data
# ============================================================

# Define a function that maps an IP (as an integer) to a country using the IP ranges
def map_ip_to_country(ip_int):
    # Find the row in ip_country_df where ip_int falls between the lower and upper bounds
    row = ip_country_df[(ip_country_df['lower_bound_ip_address'] <= ip_int) &
                        (ip_country_df['upper_bound_ip_address'] >= ip_int)]
    if not row.empty:
        return row.iloc[0]['country']
    else:
        return np.nan

# Create a new column 'country' in fraud_df by applying the mapping function
fraud_df['country'] = fraud_df['ip_int'].apply(map_ip_to_country)

# Save the merged DataFrame to a CSV file named 'merged_ip.csv'
fraud_df.to_csv("merged_ip.csv", index=False)

print("Merged file saved as merged_ip.csv")


Merged file saved as merged_ip.csv


In [12]:

# ============================================================
# STEP 7: Feature Engineering - Time-Based Features
# ============================================================

# Extract the hour of day and day of week from the purchase_time column
fraud_df['purchase_hour'] = fraud_df['purchase_time'].dt.hour
fraud_df['purchase_dayofweek'] = fraud_df['purchase_time'].dt.dayofweek


In [13]:
# ============================================================
# STEP 8: Normalize a Key Feature (purchase_value)
# ============================================================

# Using MinMaxScaler to normalize the 'purchase_value' column (assumed to be the purchase amount)
scaler = MinMaxScaler()
# Reshape is required because scaler expects a 2D array
fraud_df['purchase_value_scaled'] = scaler.fit_transform(fraud_df[['purchase_value']])


In [14]:
# ============================================================
# STEP 9: Exploratory Data Analysis (EDA) - Quick Look
# ============================================================

# Print a summary of the fraud dataset to inspect data types and new columns
print("\nFraud Data Info:")
print(fraud_df.info())

# Display the first few rows to verify the changes
print("\nFraud Data Sample:")
print(fraud_df.head())

# Optionally, you can also print summary statistics
print("\nFraud Data Summary Statistics:")
print(fraud_df.describe())



Fraud Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_id                151112 non-null  int64         
 1   signup_time            151112 non-null  datetime64[ns]
 2   purchase_time          151112 non-null  datetime64[ns]
 3   purchase_value         151112 non-null  int64         
 4   device_id              151112 non-null  object        
 5   source                 151112 non-null  object        
 6   browser                151112 non-null  object        
 7   sex                    151112 non-null  object        
 8   age                    151112 non-null  int64         
 9   ip_address             151112 non-null  float64       
 10  class                  151112 non-null  int64         
 11  ip_int                 0 non-null       float64       
 12  country                0 n

In [15]:
# ============================================================
# STEP 10: (Optional) Quick EDA on Credit Card Data
# ============================================================

print("\nCredit Card Data Info:")
print(creditcard_df.info())
print("\nCredit Card Data Sample:")
print(creditcard_df.head())


Credit Card Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7973 entries, 0 to 7972
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    7973 non-null   int64  
 1   V1      7973 non-null   float64
 2   V2      7973 non-null   float64
 3   V3      7973 non-null   float64
 4   V4      7973 non-null   float64
 5   V5      7973 non-null   float64
 6   V6      7973 non-null   float64
 7   V7      7973 non-null   float64
 8   V8      7973 non-null   float64
 9   V9      7973 non-null   float64
 10  V10     7973 non-null   float64
 11  V11     7973 non-null   float64
 12  V12     7973 non-null   float64
 13  V13     7973 non-null   float64
 14  V14     7973 non-null   float64
 15  V15     7972 non-null   float64
 16  V16     7972 non-null   float64
 17  V17     7972 non-null   float64
 18  V18     7972 non-null   float64
 19  V19     7972 non-null   float64
 20  V20     7972 non-null   float64
 21  V21     7972 

In [16]:
# Save the cleaned and merged Fraud Data with IP-to-Country information
fraud_df.to_csv("cleaned_fraud_data.csv", index=False)
print("Cleaned fraud data saved as 'cleaned_fraud_data.csv'.")

# Save the cleaned Credit Card Data as well
creditcard_df.to_csv("cleaned_creditcard_data.csv", index=False)
print("Cleaned credit card data saved as 'cleaned_creditcard_data.csv'.")

# (Optional) Save the cleaned IP-to-Country data if needed
ip_country_df.to_csv("cleaned_ip_country_data.csv", index=False)
print("Cleaned IP-to-Country data saved as 'cleaned_ip_country_data.csv'.")


Cleaned fraud data saved as 'cleaned_fraud_data.csv'.
Cleaned credit card data saved as 'cleaned_creditcard_data.csv'.
Cleaned IP-to-Country data saved as 'cleaned_ip_country_data.csv'.
